In [1]:
# Load Libraries
import warnings
from datetime import datetime

import tensorflow as tf
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Bidirectional
from keras.layers import Embedding, GRU, Input
from keras.layers.core import Dense, Dropout
from keras.models import Sequential, Model
from keras.optimizers import Adam
from model_evaluator import Evaluator
from model_preprocessor import Preprocessor

warnings.filterwarnings("ignore")
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
K.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
def bigru(max_len=67, emb_dim=32, max_vocab_len=39, W_reg=regularizers.l2(1e-4)):
    """BiGRU model with the Keras Sequential model"""
    input = Input(shape(max_len,), dtype='int32', name='bigru_input')
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, W_regularizer=W_reg)(input)
    emb = Dropout(0.5)(emb)
    bigru = Bidirectional(GRU(128))(emb)
    bigru = Dropout(0.5)(bigru)
    output = Dense(21, activation='softmax')(bigru)
    model = Model(input=[input], output=[output])
    return model

In [ ]:
# Load data using model preprocessor
x_train, x_test, y_train, y_test = Preprocessor.load_data()

In [ ]:
# Define Deep Learning Model
model_name = "BIGRU"
model = bigru()

''' Training phrase '''
epochs = 5
batch_size = 64
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy(),
                       Evaluator.precision, Evaluator.recall, Evaluator.fmeasure])

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.11)

In [ ]:
model_json = model.to_json()
with open("./saved_model/bigru.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./saved_model/bigru.h5")